# Get Last Execution

In [1]:
from IPython import get_ipython

In [2]:
session = get_ipython()

In [3]:
session.last_execution_result

<ExecutionResult object at 2b25fdd69d0, execution_count=2 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 2b25fdd6880, raw_cell="session = get_ipython()" store_history=True silent=False shell_futures=True> result=None>

In [4]:
from IPython.core.magics import ExecutionMagics

# Create A Magic

In [8]:
# This code can be put in any Python module, it does not require IPython
# itself to be running already.  It only creates the magics subclass but
# doesn't instantiate it yet.
from __future__ import print_function
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

# The class MUST call this class decorator at creation time
@magics_class
class MyMagics(Magics):

    @line_magic
    def lmagic(self, line):
        "my line magic"
        print("Full access to the main IPython object:", self.shell)
        print("Variables in the user namespace:", list(self.shell.user_ns.keys()))
        return line

    @cell_magic
    def cmagic(self, line, cell):
        "my cell magic"
        return line, cell

    @line_cell_magic
    def lcmagic(self, line, cell=None):
        "Magic that works both as %lcmagic and as %%lcmagic"
        if cell is None:
            print("Called as line magic")
            return line
        else:
            print("Called as cell magic")
            return line, cell


# In order to actually use these magics, you must register them with a
# running IPython.

def load_ipython_extension(ipython):
    """
    Any module file that define a function named `load_ipython_extension`
    can be loaded via `%load_ext module.path` or be configured to be
    autoloaded by IPython at startup time.
    """
    # You can register the class itself without instantiating it.  IPython will
    # call the default constructor on it.
    ipython.register_magics(MyMagics)

In [ ]:
%load_ext MyMagics

In [16]:
from IPython import get_ipython
get_ipython().register_magics(MyMagics)

Object `register_magics` not found.


In [1]:
%mymagics

UsageError: Line magic function `%mymagics` not found.


In [2]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %lcmagic  %ldir  %less  %lmagic  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmagic  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%lcmagic  %%mar

In [4]:
%lmagic

Full access to the main IPython object: <ipykernel.zmqshell.ZMQInteractiveShell object at 0x000002958C96E850>
Variables in the user namespace: ['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', 'print_function', 'Magics', 'magics_class', 'line_magic', 'cell_magic', 'line_cell_magic', 'MyMagics', 'load_ipython_extension', '_i', '_ii', '_iii', '_i1', '_i2', '_2', '_i3', '_3', '_i4']


''

### Access The Variables

In [5]:
number = 1

In [ ]:
locals()

In [ ]:
globals()

In [8]:
vectice_num = locals()["number"]

In [9]:
vectice_num

1

In [13]:
import io
from nbformat import read

def print_cell_code(fname, cellno):
    with io.open(fname, 'r', encoding='utf-8') as f:
        nb = read(f, 4)
    cell = nb.cells[cellno]
    print(cell.source)

print_cell_code("Untitled.ipynb",13)

number = 1


# Get Cell Information
- The cell number and contents
- https://ipython.readthedocs.io/en/stable/api/generated/IPython.core.events.html#IPython.core.events.pre_run_cell
- stacks so events registered need to be cleaned up
- There is pre_execute, pre_run_cell, post_execute, post_run_cell 

In [2]:
# This captures the code called, we then use regex to identify the variables in locals. The variables found can then be logged to vectice.
import IPython
ip = IPython.get_ipython()
global run 
run = 0
def f():
    global cell_content
    global cell_info
    global run
    if run < 2:
        cell_content = ip.get_parent()['content']['code']
        cell_info = ip.get_parent()
        run += 1
        
ip.events.register('post_run_cell', f)

In [1]:
from autolog.autolog import autolog

In [2]:
# A vectice.autolog() method would be called here to invoke the above even to be registered and we capture the local variables
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
import pandas as pd

params = {
        "C": 1e5,
        "class_weight": None,
        "dual": False,
        "fit_intercept": True,
        "intercept_scaling": 1,
        "l1_ratio": None,
        "max_iter": 100,
        "multi_class": "auto",
        "n_jobs": None,
        "penalty": "l2",
        "random_state": 0,
        "solver": "lbfgs",
        "tol": 0.0001,
        "verbose": 0,
        "warm_start": False,
    }
iris = load_iris()
dummy_df = pd.DataFrame(iris.data)
x = iris.data[:, :2]  # we only take the first two features.
y = iris.target

logreg = LogisticRegression(**params).fit(x, y)
autolog(locals())

{'getOrgConfig': {'organization': {'configuration': {'dfStatisticsRowThreshold': 100}}, '__typename': 'OrgConfigOutput'}}


Welcome, anell. You`re now successfully connected to Vectice.

To access your personal workspace, use connection.my_workspace
To access a specific workspace, use connection.workspace(Workspace ID)
To get a list of workspaces you can access and their IDs, use connection.list_workspaces()

If you are using a notebook you can call the help by using a Vectice returned object with the builtin notebook "?":
>> connection?

If you are using an IDE you can call the help() method on any object returned by Vectice:
>> help(connection)

For quick access to the list of workspaces in the Vectice web app, visit:
https://app.vectice.com/workspaces
Phase 'log test 10' successfully retrieved."

For quick access to the Phase in the Vectice web app, visit:
https://app.vectice.com/browse/phase/PHA-3661
Iteration number 1 successfully retrieved.

For quick access to the Iteration in the Vectice web app, visit:
https://app.vectice.com/browse/iteration/ITR-1486
Model LogisticRegression successfully attached 

In [4]:
type(x)

C:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


numpy.ndarray

In [5]:
import re
matches = re.findall(r'(.*?) =', cell_content)
matches

['params', 'iris', 'dummy_df', 'x', 'y', 'logreg']

In [6]:
local_vars = locals() # calling locals in the list comprehension causes inconsistencies 
vectice_data = {variable: local_vars[variable] for variable in matches}

In [ ]:
vectice_data

In [9]:
print(vectice_data['logreg'])
print(vectice_data['dummy_df'])

LogisticRegression(C=100000.0, random_state=0)
       0    1    2    3
0    5.1  3.5  1.4  0.2
1    4.9  3.0  1.4  0.2
2    4.7  3.2  1.3  0.2
3    4.6  3.1  1.5  0.2
4    5.0  3.6  1.4  0.2
..   ...  ...  ...  ...
145  6.7  3.0  5.2  2.3
146  6.3  2.5  5.0  1.9
147  6.5  3.0  5.2  2.0
148  6.2  3.4  5.4  2.3
149  5.9  3.0  5.1  1.8

[150 rows x 4 columns]


In [9]:
vectice_data['logreg']

LogisticRegression(C=100000.0, random_state=0)

In [10]:
import copy
def get_model(vectice_data):
    for key in vectice_data.keys():
        if "sklearn" and "model" in str(vectice_data[key].__class__):
            return copy.deepcopy(vectice_data[key])
    return None

In [11]:
sklearn_model = get_model(vectice_data)

In [12]:
sklearn_model.get_params()

{'C': 100000.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 0,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [11]:
from pandas import DataFrame

In [12]:
def get_df(vectice_data):
    for key in vectice_data.keys():
        if isinstance(vectice_data[key], DataFrame):
            return copy.deepcopy(vectice_data[key])
    return None

In [13]:
pandas_df = get_df(vectice_data)

In [17]:
pandas_df.head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [16]:
sklearn_model

LogisticRegression(C=100000.0, random_state=0)

In [ ]:
# This will unregister our event and prevent stacking 
ip.events.unregister('post_run_cell', f)

In [1]:
from autolog import _identify_assets

oh no


ImportError: cannot import name '_identify_assets' from partially initialized module 'autolog' (most likely due to a circular import) (C:\Users\Aidan\autolog\autolog\__init__.py)